In [ ]:
import numpy as np
# import argparse
import os
import sys
import torch
import torch.fft as F
from importlib import reload
# from torch.nn.functional import /relu
import torch.nn as nn
# import torch.nn.functional as Func
# import torch.optim as optim
import utils
from utils import *
from loupe_env.loupe_wrap import LOUPE
from MoDL.MoDL import MoDL
from matplotlib import pyplot as plt
import random
import copy
# from sklearn.model_selection import train_test_split

# import skimage
from mnet.mnet_v2 import MNet
from unet.unet_model import UNet
from unet.unet_model_fbr import Unet
from unet.unet_model_banding_removal_fbr import UnetModel
# from mask_backward_v4 import *
# from sigpy.mri.app import TotalVariationRecon

torch.manual_seed(0)
np.random.seed(0)
random.seed(0)

In [ ]:
testdata = torch.tensor(np.load('/mnt/shared_a/fastMRI/knee_singlecoil_train.npz')['data'])
print('test data shape: ',testdata.shape)

## make figures

### accuracy plots

In [ ]:
dir_rec1 = '/mnt/shared_a/checkpoints/leo/mri/8fold_mnetunet_direct.npz'
dir_rec2 = '/mnt/shared_a/checkpoints/leo/mri/8fold_mnetunet.npz'
dir_rec3 = '/mnt/shared_a/checkpoints/leo/mri/8fold_mnetmodl.npz'
dir_rec4 = '/mnt/shared_a/checkpoints/leo/mri/8fold_loupe.npz'
dir_rec5 = '/mnt/shared_a/checkpoints/leo/mri/8fold_rand.npz'
dir_rec6 = '/mnt/shared_a/checkpoints/leo/mri/8fold_equidist.npz'

In [ ]:
rec1 = np.load(dir_rec1)
l1err_mnetunet_dir,l2err_mnetunet_dir,hfens_mnetunet_dir,ssims_mnetunet_dir,psnrs_mnetunet_dir = \
rec1['l1err'], rec1['l2err'], rec1['hfens'], rec1['ssims'], rec1['psnrs']

rec2 = np.load(dir_rec2)
l1err_mnetunet,l2err_mnetunet,hfens_mnetunet,ssims_mnetunet,psnrs_mnetunet = \
rec2['l1err'], rec2['l2err'], rec2['hfens'], rec2['ssims'], rec2['psnrs']

rec3 = np.load(dir_rec3)
l1err_mnetmodl,l2err_mnetmodl,hfens_mnetmodl,ssims_mnetmodl,psnrs_mnetmodl = \
rec3['l1err'], rec3['l2err'], rec3['hfens'], rec3['ssims'], rec3['psnrs']

rec4 = np.load(dir_rec4)
l1err_loupe,l2err_loupe,hfens_loupe,ssims_loupe,psnrs_loupe = \
rec4['l1err'], rec4['l2err'], rec4['hfens'], rec4['ssims'], rec4['psnrs']

rec5 = np.load(dir_rec5)
l1err_rand,l2err_rand,hfens_rand,ssims_rand,psnrs_rand = \
rec5['l1err'], rec5['l2err'], rec5['hfens'], rec5['ssims'], rec5['psnrs']

rec6 = np.load(dir_rec6)
l1err_equidist,l2err_equidist,hfens_equidist,ssims_equidist,psnrs_equidist = \
rec6['l1err'], rec6['l2err'], rec6['hfens'], rec6['ssims'], rec6['psnrs']


In [ ]:
# Figure: Accuracy comparison

figsize=(29,17)
label_fontsize = 26
xtick_fontsize  = 19
ytick_fontsize  = 26
rotation_angle = 19
nrows=2; 
ncols=3

plt.clf()
fig, axs = plt.subplots(nrows=nrows, ncols=ncols, sharex=False, figsize=figsize)
labels = ['MNet-Unet-co', 'MNet-Unet-sep','MNet-MoDL','LOUPE','Rand.-Unet', 'Equidist.-Unet']

data = [l1err_mnetunet_dir,l1err_mnetunet,l1err_mnetmodl,l1err_loupe,l1err_rand,l1err_equidist]
axs[0,0].boxplot(data,notch=True,showmeans=True)
axs[0,0].set_xticklabels(labels,fontsize=label_fontsize)
axs[0,0].set_ylabel(r"Rel. $\ell_1$ error",fontsize=label_fontsize)
axs[0,0].set_ylim([0, 0.5])
axs[0,0].tick_params(axis='x', which='major', labelsize=xtick_fontsize,rotation=rotation_angle)
axs[0,0].tick_params(axis='y', which='major', labelsize=ytick_fontsize,rotation=0)

data = [l2err_mnetunet_dir,l2err_mnetunet,l2err_mnetmodl,l2err_loupe,l2err_rand,l2err_equidist]
axs[0,1].boxplot(data,notch=True,showmeans=True)
axs[0,1].set_xticklabels(labels,fontsize=label_fontsize)
axs[0,1].set_ylabel(r"Rel. $\ell_2$ error",fontsize=label_fontsize)
axs[0,1].set_ylim([0, 0.5])
axs[0,1].tick_params(axis='x', which='major', labelsize=xtick_fontsize,rotation=rotation_angle)
axs[0,1].tick_params(axis='y', which='major', labelsize=ytick_fontsize,rotation=0)

data = [hfens_mnetunet_dir,hfens_mnetunet,hfens_mnetmodl,hfens_loupe,hfens_rand,hfens_equidist]
axs[0,2].boxplot(data,notch=True,showmeans=True)
axs[0,2].set_xticklabels(labels,fontsize=label_fontsize)
axs[0,2].set_ylabel(r"HFEN",fontsize=label_fontsize)
axs[0,2].set_ylim([0, 1])
axs[0,2].tick_params(axis='x', which='major', labelsize=xtick_fontsize,rotation=rotation_angle)
axs[0,2].tick_params(axis='y', which='major', labelsize=ytick_fontsize,rotation=0)

data = [ssims_mnetunet_dir,ssims_mnetunet,ssims_mnetmodl,ssims_loupe,ssims_rand,ssims_equidist]
axs[1,0].boxplot(data,notch=True,showmeans=True)
axs[1,0].set_xticklabels(labels,fontsize=label_fontsize)
axs[1,0].set_ylabel(r"SSIM",fontsize=label_fontsize)
axs[1,0].tick_params(axis='x', which='major', labelsize=xtick_fontsize,rotation=rotation_angle)
axs[1,0].tick_params(axis='y', which='major', labelsize=ytick_fontsize,rotation=0)

data = [psnrs_mnetunet_dir/2,psnrs_mnetunet/2,psnrs_mnetmodl/2,psnrs_loupe/2,psnrs_rand/2,psnrs_equidist/2]
axs[1,1].boxplot(data,notch=True,showmeans=True)
axs[1,1].set_xticklabels(labels,fontsize=label_fontsize)
axs[1,1].set_ylabel(r"PSNR",fontsize=label_fontsize)
axs[1,1].tick_params(axis='x', which='major', labelsize=xtick_fontsize,rotation=rotation_angle)
axs[1,1].tick_params(axis='y', which='major', labelsize=ytick_fontsize,rotation=0)

axs[-1, -1].axis('off')
fig_curr = plt.gcf()
plt.show()
plt.draw()
dir_fig = '/home/huangz78/mri/figures/accuracy_8fold.eps'
fig_curr.savefig(dir_fig,format='eps',dpi=600,transparent=True,bbox_inches='tight')

### example plots

In [ ]:
# specify sampling parameters
heg = 320
wid = 320
sparsity = .125
preselect_num = 8
budget = int(heg * sparsity - preselect_num)

print(f'base corefreq is {preselect_num}, sampling budget is {budget}')

# load mnet
mnet = MNet(beta=1,in_chans=2,out_size=320-preselect_num, imgsize=(heg,wid),poolk=3)
mnetpath = '/mnt/shared_a/checkpoints/leo/mri/mnet_v2_split_trained_cf_8_bg_32_unet_in_chan_1_epoch9.pt'

checkpoint = torch.load(mnetpath)
mnet.load_state_dict(checkpoint['model_state_dict'])
# mnet.apply(mnet_weights_init)
mnet.eval()
print('MNet loaded successfully from: ' + mnetpath)

model_in_chans = 2
device = torch.device('cuda:0')
for ind in range(testdata.shape[0]):
    testdata[ind,:,:] = testdata[ind,:,:]/torch.max(torch.abs(testdata[ind,:,:]))
print('test data size:', testdata.shape)
test_in, mask_in = mnet_getinput(mnet,testdata,base=preselect_num,budget=budget,batchsize=10,\
                                  unet_channels=model_in_chans,return_mask=True,device=device)

In [ ]:
imgind = np.random.randint(test_in.shape[0])
print(f'current image index: {imgind}')

In [ ]:
## Unet-co
unet_inchannels = 2
unet_recon = Unet(in_chans=unet_inchannels,out_chans=1,chans=64)
unetpath = '/mnt/shared_a/checkpoints/leo/recon/unet_fbr_2_chans_64_epoch_39.pt'

checkpoint = torch.load(unetpath)
unet_recon.load_state_dict(checkpoint['model_state_dict'])
print('Unet loaded successfully from: ' + unetpath )
unet_recon.eval()
print('nn\'s are ready')

In [ ]:
img = test_in[imgind]
recon_unet_co = unet_recon(img.unsqueeze(0)).detach()
recon_unet_co.shape

In [ ]:
plt.figure()
plt.imshow(img[0,:,:])
plt.colorbar()

In [ ]:
figsize=(29,17)

nrows=2 
ncols=3

plt.clf()
fig, axs = plt.subplots(nrows=nrows, ncols=ncols, sharex=False, figsize=figsize)


## evaluate MNet - UNet

In [ ]:
# specify sampling parameters
heg = 320
wid = 320
sparsity = .125
preselect_num = 8
budget = int(heg * sparsity - preselect_num)

print(f'base corefreq is {preselect_num}, sampling budget is {budget}')

# load mnet
mnet = MNet(beta=1,in_chans=2,out_size=320-preselect_num, imgsize=(heg,wid),poolk=3)
mnetpath = '/mnt/shared_a/checkpoints/leo/mri/mnet_v2_split_trained_cf_8_bg_32_unet_in_chan_1_epoch9.pt'

checkpoint = torch.load(mnetpath)
mnet.load_state_dict(checkpoint['model_state_dict'])
# mnet.apply(mnet_weights_init)
mnet.eval()
print('MNet loaded successfully from: ' + mnetpath)

# load unet
unet_inchannels = 2
unet_recon = Unet(in_chans=unet_inchannels,out_chans=1,chans=64)
unetpath = '/mnt/shared_a/checkpoints/leo/recon/unet_fbr_2_chans_64_epoch_39.pt'

checkpoint = torch.load(unetpath)
unet_recon.load_state_dict(checkpoint['model_state_dict'])
print('Unet loaded successfully from: ' + unetpath )
unet_recon.eval()
print('nn\'s are ready')


In [ ]:
l1err,l2err,hfens,ssims,psnrs = mnet_eval(testdata,mnet,unet_recon,\
                                          base=preselect_num,budget=budget,batchsize=10,device='cuda:0')

plt.boxplot(ssims.numpy())
plt.title(f'SSIM mean={ssims.numpy().mean():.4f}')
plt.show()

plt.boxplot(psnrs.numpy())
plt.title(f'PSNR mean={psnrs.numpy().mean():.4f}')
plt.show()

plt.boxplot(l1err.numpy())
plt.title(f'l1err mean={l1err.numpy().mean():.4f}')
plt.show()

plt.boxplot(l2err.numpy())
plt.title(f'l2err mean={l2err.numpy().mean():.4f}')
plt.show()

plt.boxplot(hfens.numpy())
plt.title(f'HFEN mean={hfens.numpy().mean():.4f}')
plt.show()

filename = '/mnt/shared_a/checkpoints/leo/mri/8fold_mnetunet.npz'
np.savez(filename,l1err=l1err,l2err=l2err,hfens=hfens,ssims=ssims,psnrs=psnrs)

In [ ]:
# specify sampling parameters
heg = 320
wid = 320
sparsity = .125
preselect_num = 8
budget = int(heg * sparsity - preselect_num)

print(f'base corefreq is {preselect_num}, sampling budget is {budget}')

# load mnet
mnet = MNet(beta=1,in_chans=2,out_size=320-preselect_num, imgsize=(heg,wid),poolk=3)
mnetpath = '/mnt/shared_a/checkpoints/leo/mri/mnet_v2_split_trained_cf_8_bg_32_unet_in_chan_1_epoch9.pt'

checkpoint = torch.load(mnetpath)
mnet.load_state_dict(checkpoint['model_state_dict'])
# mnet.apply(mnet_weights_init)
mnet.eval()
print('MNet loaded successfully from: ' + mnetpath)

# load unet
unet_recon = unet = UNet(in_chans=1,n_classes=1,bilinear=False,skip=True)
unetpath = '/mnt/shared_a/checkpoints/leo/mri/unet_v2_split_trained_cf_8_bg_32_unet_in_chan_1_epoch9.pt'

checkpoint = torch.load(unetpath)
unet_recon.load_state_dict(checkpoint['model_state_dict'])
print('Unet loaded successfully from: ' + unetpath )
unet_recon.eval()
print('nn\'s are ready')

In [ ]:
l1err,l2err,hfens,ssims,psnrs = mnet_eval(testdata,mnet,unet_recon,\
                                          base=preselect_num,budget=budget,batchsize=10,device='cuda:0')

plt.boxplot(ssims.numpy())
plt.title(f'SSIM mean={ssims.numpy().mean():.4f}')
plt.show()

plt.boxplot(psnrs.numpy())
plt.title(f'PSNR mean={psnrs.numpy().mean():.4f}')
plt.show()

plt.boxplot(l1err.numpy())
plt.title(f'l1err mean={l1err.numpy().mean():.4f}')
plt.show()

plt.boxplot(l2err.numpy())
plt.title(f'l2err mean={l2err.numpy().mean():.4f}')
plt.show()

plt.boxplot(hfens.numpy())
plt.title(f'HFEN mean={hfens.numpy().mean():.4f}')
plt.show()

filename = '/mnt/shared_a/checkpoints/leo/mri/8fold_mnetunet_direct.npz'
np.savez(filename,l1err=l1err,l2err=l2err,hfens=hfens,ssims=ssims,psnrs=psnrs)

## evaluate MNet - MoDL

In [ ]:
modl = MoDL(in_chans=2,out_chans=2,chans=32,num_pool_layers=4,drop_prob=0)
modlpath = '/mnt/shared_a/checkpoints/leo/recon/MoDL_2_chans_32_epoch_39.pt'
checkpoint = torch.load(modlpath)
modl.load_state_dict(checkpoint['model_state_dict'])
print(f'MoDL model loaded successfully from {modlpath}')

In [ ]:
l1err,l2err,hfens,ssims,psnrs = mnet_eval(testdata,mnet,modl,\
                                          base=preselect_num,budget=budget,batchsize=5,device='cuda:0')

plt.boxplot(ssims.numpy())
plt.title(f'SSIM mean={ssims.numpy().mean():.4f}')
plt.show()

plt.boxplot(psnrs.numpy())
plt.title(f'PSNR mean={psnrs.numpy().mean():.4f}')
plt.show()

plt.boxplot(l1err.numpy())
plt.title(f'l1err mean={l1err.numpy().mean():.4f}')
plt.show()

plt.boxplot(l2err.numpy())
plt.title(f'l2err mean={l2err.numpy().mean():.4f}')
plt.show()

plt.boxplot(hfens.numpy())
plt.title(f'HFEN mean={hfens.numpy().mean():.4f}')
plt.show()

filename = '/mnt/shared_a/checkpoints/leo/mri/8fold_mnetmodl.npz'
np.savez(filename,l1err=l1err,l2err=l2err,hfens=hfens,ssims=ssims,psnrs=psnrs)

## evaluate baseline --- rand and equidist

In [ ]:
# load unet
unet_inchannels = 2
unet_recon = Unet(in_chans=unet_inchannels,out_chans=1,chans=64)
unetpath = '/mnt/shared_a/checkpoints/leo/recon/unet_fbr_2_chans_64_rand_epoch_38.pt'

checkpoint = torch.load(unetpath)
unet_recon.load_state_dict(checkpoint['model_state_dict'])
print('Unet loaded successfully from: ' + unetpath )
unet_recon.eval()
print('nn\'s are ready')

In [ ]:
l1err,l2err,hfens,ssims,psnrs = baseline_eval(testdata,model=unet_recon,mode='rand',\
                                          base=preselect_num,budget=budget,batchsize=10,device='cuda:0')

plt.boxplot(ssims.numpy())
plt.title(f'SSIM mean={ssims.numpy().mean():.4f}')
plt.show()

plt.boxplot(psnrs.numpy())
plt.title(f'PSNR mean={psnrs.numpy().mean():.4f}')
plt.show()

plt.boxplot(l1err.numpy())
plt.title(f'l1err mean={l1err.numpy().mean():.4f}')
plt.show()

plt.boxplot(l2err.numpy())
plt.title(f'l2err mean={l2err.numpy().mean():.4f}')
plt.show()

plt.boxplot(hfens.numpy())
plt.title(f'HFEN mean={hfens.numpy().mean():.4f}')
plt.show()

filename = '/mnt/shared_a/checkpoints/leo/mri/8fold_rand.npz'
np.savez(filename,l1err=l1err,l2err=l2err,hfens=hfens,ssims=ssims,psnrs=psnrs)

In [ ]:
# load unet
unet_inchannels = 2
unet_recon = Unet(in_chans=unet_inchannels,out_chans=1,chans=64)
unetpath = '/mnt/shared_a/checkpoints/leo/recon/unet_fbr_2_chans_64_equidist_epoch_39.pt'

checkpoint = torch.load(unetpath)
unet_recon.load_state_dict(checkpoint['model_state_dict'])
print('Unet loaded successfully from: ' + unetpath )
unet_recon.eval()
print('nn\'s are ready')

In [ ]:
l1err,l2err,hfens,ssims,psnrs = baseline_eval(testdata,model=unet_recon,mode='rand',\
                                          base=preselect_num,budget=budget,batchsize=10,device='cuda:0')

plt.boxplot(ssims.numpy())
plt.title(f'SSIM mean={ssims.numpy().mean():.4f}')
plt.show()

plt.boxplot(psnrs.numpy())
plt.title(f'PSNR mean={psnrs.numpy().mean():.4f}')
plt.show()

plt.boxplot(l1err.numpy())
plt.title(f'l1err mean={l1err.numpy().mean():.4f}')
plt.show()

plt.boxplot(l2err.numpy())
plt.title(f'l2err mean={l2err.numpy().mean():.4f}')
plt.show()

plt.boxplot(hfens.numpy())
plt.title(f'HFEN mean={hfens.numpy().mean():.4f}')
plt.show()

filename = '/mnt/shared_a/checkpoints/leo/mri/8fold_equidist.npz'
np.savez(filename,l1err=l1err,l2err=l2err,hfens=hfens,ssims=ssims,psnrs=psnrs)

## evaluate LOUPE

In [ ]:
# load loupe model
loupepath = '/mnt/shared_a/checkpoints/leo/mri/loupe_8fold_base_8_epoch_39.pt'
loupe = LOUPE(in_chans=1,unet_skip=False,shape=[heg,wid],sparsity=sparsity,\
                  preselect=True,preselect_num=preselect_num)

checkpoint = torch.load(loupepath)
loupe.load_state_dict(checkpoint['model_state_dict'])
loupe.eval()
print(f'loupe model is successfully imported from : {loupepath}')


In [ ]:
l1err,l2err,hfens,ssims,psnrs = loupe_eval(loupe,testdata,\
                                           preselect_num=preselect_num,sparsity=sparsity,\
                                           batchsize=10,device=torch.device('cuda:0'))

plt.boxplot(ssims.numpy())
plt.title(f'SSIM mean={ssims.numpy().mean():.4f}')
plt.show()

plt.boxplot(psnrs.numpy())
plt.title(f'PSNR mean={psnrs.numpy().mean():.4f}')
plt.show()

plt.boxplot(l1err.numpy())
plt.title(f'l1err mean={l1err.numpy().mean():.4f}')
plt.show()

plt.boxplot(l2err.numpy())
plt.title(f'l2err mean={l2err.numpy().mean():.4f}')
plt.show()

plt.boxplot(hfens.numpy())
plt.title(f'HFEN mean={hfens.numpy().mean():.4f}')
plt.show()

filename = '/mnt/shared_a/checkpoints/leo/mri/8fold_loupe.npz'
np.savez(filename,l1err=l1err,l2err=l2err,hfens=hfens,ssims=ssims,psnrs=psnrs)